In [1]:
# Calculate fission Spectra weighting 
import csv    # Library to read & process csv-formatted files
import sys    # Library to parse command-line arguments
import os
from math import sqrt, sinh, pi, exp
import numpy as np

rootdir=os.path.abspath(os.getcwd())

# Read in the spectra from file
fname = rootdir+"/E_fiss.csv"   # Fission spectra input file
e_bins=[]
fiss=[]
err=[]
try: 
    ifile = open(fname, 'r') 

    # Store the spectrum
    bins=[]
    for line in ifile:
        split_list=line.split(',')
        e_bins.append(float(split_list[0].strip()))
        fiss.append(float(split_list[1].strip()))
        err.append(float(split_list[2].strip()))

    # Close the file
    ifile.close()
except IOError as e:
    print "I/O error({0}): {1}".format(e.errno, e.strerror)
    print "File not found was: {0}".format(fname)
    
np.asarray(fiss)
fiss=fiss/np.sum(fiss)

In [2]:
# Read in the bin structure from file
rootdir_235=os.path.abspath(os.path.join(os.path.abspath(os.path.join(rootdir, os.pardir)), os.pardir))+'/Validation/GEFY_5.3/235U/ChainYields'
rootdir_238=os.path.abspath(os.path.join(os.path.abspath(os.path.join(rootdir, os.pardir)), os.pardir))+'/Validation/GEFY_5.3/238U/ChainYields'
fname = rootdir+"/Bins.csv"   # Bin structure input file
bins=[]

try: 
    ifile = open(fname, 'r') 
    
    # Skip header line
    line=ifile.next()

    # Store the spectrum
    bins=[]
    for line in ifile:
        split_list=line.split(',')
        bins.append(float(split_list[2].strip()))

    # Close the file
    ifile.close()
except IOError as e:
    print "I/O error({0}): {1}".format(e.errno, e.strerror)
    print "File not found was: {0}".format(fname)

#print len(bins)
#print bins

In [26]:
import csv

def Build_Weighted_FPs(bins,fw,datadir):
    one=0
    two=0
    y={}
    err={}
    w=[]
    fy=[]

    # Use weighted fission spectra to generate mass yield curves (sum yields?)
    for subdir, dirs, files in os.walk(datadir):
        for file in files:
            #print subdir, file
            print file
            E=float(file.split('_')[0].strip())/1E6
            weight = 0
            one+=1
            try: 
                ifile = open(subdir+'/'+file, 'r') 
                reader = csv.reader(ifile)
                
                # Determine weight for given energy
                for i in range(0,len(bins)):
                    if bins[i] == E:
                        two+=1
                        weight=fw[i]
                        
                for line in reader:
                    # Build dictionaries
                    if not line[1] in y.keys():  # this is the first time we've encountered this mass A
                        # let's initialize our dictionaries
                            y[line[1]] = float(line[2])*weight
                            err[line[1]] = (float(line[3])*weight)**2
                    else:  # we've encoutered this mass before, no need to initialize, just add to it
                        y[line[1]] += float(line[2])*weight
                        err[line[1]] += (float(line[3])*weight)**2

                    if int(line[1])==160:
                        w.append(weight)
                        fy.append(float(line[2]))
                        print float(line[3])

                # Close the file
                ifile.close()
            except IOError as e:
                print "I/O error({0}): {1}".format(e.errno, e.strerror)
                print "File not found was: {0}".format(fname)
    #print y
    #print y.keys()

    if one!=two or one!=len(bins):
        print "/n/nError: Not all of the files from {} were imported correctly./n/n".format(datadir)
        print one, two, len(bins)
        
    return y, err

In [27]:
#y={}
#err={}
#(y,err)=Build_Weighted_FPs(bins,fiss,rootdir_235)

y_235={}
err_235={}
f_235=0.93155
(y_235,err_235)=Build_Weighted_FPs(bins,fiss*f_235,rootdir_235)

print "\n\n"
for A in y_235.keys():
    err[A]=sqrt(err_235[A])

#y_238={}
#err_238={}
#f_238=1-f_235
#(y_238,err_238)=Build_Weighted_FPs(bins,fiss*f_238,rootdir_238)

#for A in y_238.keys():
#    err[A]=sqrt(err_238[A])

#for A in y.keys():
#    try:
#        y[A]=y_235[A]+y_238[A]
#        err[A]=sqrt(err_235[A]**2+err_238[A]**2)
#    except KeyError as e:
#        print "Key error: {0} not found for both isotopes.".format(A)

1000000.0_u235_gefy_chain.csv
Energy = 1.0
Fractional weight= 0.0389226361101
2.82681799909e-05
600000.0_u235_gefy_chain.csv
Energy = 0.6
Fractional weight= 0.0353027557455
7.74596669241e-06
800000.0_u235_gefy_chain.csv
Energy = 0.8
Fractional weight= 0.0279128730647
1.50665191733e-05
8000000.0_u235_gefy_chain.csv
Energy = 8.0
Fractional weight= 0.0044952743578
7.15925973827e-05
2500000.0_u235_gefy_chain.csv
Energy = 2.5
Fractional weight= 0.0205277016631
5.26909868953e-05
6000000.0_u235_gefy_chain.csv
Energy = 6.0
Fractional weight= 0.00882168926196
4.92544414241e-05
12500000.0_u235_gefy_chain.csv
Energy = 12.5
Fractional weight= 0.00571805078246
7.05955381026e-05
300000.0_u235_gefy_chain.csv
Energy = 0.3
Fractional weight= 0.0632237376671
6.40312423743e-06
4000000.0_u235_gefy_chain.csv
Energy = 4.0
Fractional weight= 0.0191791197079
7.76646637796e-05
400000.0_u235_gefy_chain.csv
Energy = 0.4
Fractional weight= 0.0752364680566
6.92820323028e-06
14000000.0_u235_gefy_chain.csv
Energy = 

TypeError: list indices must be integers, not str

In [5]:
# Output the result
outfn="ETA_sum_fy.csv"

def writeout(outfn,outl):  # function to write our output in csv format
    ofile = open(outfn,"w")

    for out in outl:
        ofile.write(str(out[0])
                    + "," + str(out[1])
                    + "," + str(out[2])
                    + "," + str(out[3])
                    + "," + str(out[4])
                    + "\n")

    ofile.close()

outl=[]
for A in y.keys():
    outl.append((A,0,y[A],err[A]," "))
writeout(outfn,outl)

In [43]:
from scipy.interpolate import interp1d
f = interp1d(bins,fiss,kind='cubic')
E=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0,5.5,6.0,6.5,7.0,7.5,8.0,8.5,9.0,9.5,10.0,10.5,11.0,11.5,12.0,12.5,13.0,13.5,14.0,14.5,15.0,15.5,16.0]
np.sum(f(E))

1.0743238550175147

In [49]:
import scipy as sp
import scipy.interpolate

def log_interp1d(xx, yy, kind='linear'):
    logx = np.log10(xx)
    logy = np.log10(yy)
    lin_interp = sp.interpolate.interp1d(logx, logy, kind=kind)
    log_interp = lambda zz: np.power(10.0, lin_interp(np.log10(zz)))
    return log_interp

f = log_interp1d(bins,fiss,kind='cubic')
print np.sum(f(E))

print bins[25],fiss[25]
print f(1.27)

1.16733912729
1.27 0.0714438812603
0.0714438812603
